In [1]:
%reload_ext watermark
%matplotlib inline
import numpy as np
import pandas as pd

def rc(seq):
    """
    from http://stackoverflow.com/a/25189185/7146785
    """
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    
    rev_seq = "".join(complement.get(base, base) for base in reversed(seq))
    
    return(rev_seq)
%watermark -iv -m -h -p metapool,sample_sheet,openpyxl

metapool    : 0.0.0.dev0
sample_sheet: 0.12.0
openpyxl    : 3.0.7

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Hostname: debugatron-4000.attlocal.net

numpy : 1.18.1
pandas: 1.0.2



## Import plate map from IDT

In [46]:
row_offset = 0
col_offset = 0

print('%s%s' % (rowmod('A'), colmod('1')))
print('%s%s' % (rowmod('H'), colmod('12')))

row_offset = 0
col_offset = 1

print('%s%s' % (rowmod('A'), colmod('1')))
print('%s%s' % (rowmod('H'), colmod('12')))


row_offset = 1
col_offset = 0

print('%s%s' % (rowmod('A'), colmod('1')))
print('%s%s' % (rowmod('H'), colmod('12')))


row_offset = 1
col_offset = 1

print('%s%s' % (rowmod('A'), colmod('1')))
print('%s%s' % (rowmod('H'), colmod('12')))

A1
O23
A2
O24
B1
P23
B2
P24


In [73]:
def condense_plates(input_df, plate_names, plate_col, row_col, col_col):
    
    df = input_df.copy()

    df['condensed row'] = np.nan
    df['condensed col'] = np.nan
    
    rowmod = lambda x: chr((ord(x.upper()) - 65) * 2 + row_offset + 65)
    colmod = lambda x: int((int(x) - 1) * 2 + 1 + col_offset)
    
    if len(plate_names) > 4:
        raise ValueError('Too many plates! Provide up to 4.')

    for i, plate in enumerate(plate_names):
        row_offset = int(np.floor(i/2))
        col_offset = i % 2
        
        df.loc[df[plate_col] == plate, 'condensed row'] = df.loc[df[plate_col] == plate, row_col].apply(rowmod)
        df.loc[df[plate_col] == plate, 'condensed col'] = df.loc[df[plate_col] == plate, col_col].apply(colmod)
    
    df['condensed col'] = df['condensed col'].astype(int)
    return(df)

In [74]:
# read in IDT plate map
idt_map = pd.read_excel('./test_data/iTru/Plate_Specs_iTru5_96-1-4_iTru7_96-1-4_OnePerPlate_Dec_2017.xlsx')
idt_map.dropna(thresh=3, inplace=True)

# split row and column from 96-well format
idt_map['origin row'] = idt_map['Well Position'].str[0]
idt_map['origin col'] = idt_map['Well Position'].str[1:].astype(int)

In [92]:
# add 384-plate positions to i7 plate
idt_map_i7 = condense_plates(idt_map.loc[idt_map['Sequence Name'].str.startswith('iTru7'),],
                ['iTru7_plate_96-1 REPLICATE','iTru7_plate_96-2 REPLICATE',
                 'iTru7_plate_96-3 REPLICATE','iTru7_plate_96-4 REPLICATE'],
                'Plate Name',
                'origin row',
                'origin col').reset_index(drop=True)

# add 384-plate positions to i5 plate
idt_map_i5 = condense_plates(idt_map.loc[idt_map['Sequence Name'].str.startswith('iTru5'),],
                ['iTru5_plate_96-1 REPLICATE','iTru5_plate_96-2 REPLICATE',
                 'iTru5_plate_96-3 REPLICATE','iTru5_plate_96-4 REPLICATE'],
                'Plate Name',
                'origin row',
                'origin col').reset_index(drop=True)

In [93]:
idt_map_i5.head()

,Plate Name,Plate Barcode,Well Position,Sequence Name,Sequence,Manufacturing ID,Measured Molecular Weight,Calculated Molecular Weight,OD260,nmoles,µg,Measured Concentration µM,Final Volume µL,Extinction Coefficient L/(mole·cm),Tm,origin row,origin col,condensed row,condensed col
0,iTru5_plate_96-1 REPLICATE,11505386.0,A01,iTru5_01_A,AAT GAT ACG GCG ACC ACC GAG ATC TAC ACA CCG AC...,231436960.0,0.0,15815.0,0.62,1.25,20.0,125.0,10.0,497600.0,69.0,A,1,A,1
1,iTru5_plate_96-1 REPLICATE,11505386.0,A02,iTru5_02_A,AAT GAT ACG GCG ACC ACC GAG ATC TAC ACC TTC GC...,231436948.0,0.0,15797.0,0.61,1.25,20.0,125.0,10.0,488000.0,69.0,A,2,A,3
2,iTru5_plate_96-1 REPLICATE,11505386.0,A03,iTru5_03_A,AAT GAT ACG GCG ACC ACC GAG ATC TAC ACA ACA CC...,231436936.0,0.0,15775.0,0.62,1.25,20.0,125.0,10.0,494700.0,69.0,A,3,A,5
3,iTru5_plate_96-1 REPLICATE,11505386.0,A04,iTru5_04_A,AAT GAT ACG GCG ACC ACC GAG ATC TAC ACC GTA TC...,231436924.0,0.0,15788.0,0.61,1.25,20.0,125.0,10.0,487900.0,69.0,A,4,A,7
4,iTru5_plate_96-1 REPLICATE,11505386.0,A05,iTru5_05_A,AAT GAT ACG GCG ACC ACC GAG ATC TAC ACG GTA CG...,231436912.0,0.0,15886.0,0.63,1.25,20.0,125.0,10.0,500900.0,69.0,A,5,A,9


In [94]:
idt_map_i7.head()

,Plate Name,Plate Barcode,Well Position,Sequence Name,Sequence,Manufacturing ID,Measured Molecular Weight,Calculated Molecular Weight,OD260,nmoles,µg,Measured Concentration µM,Final Volume µL,Extinction Coefficient L/(mole·cm),Tm,origin row,origin col,condensed row,condensed col
0,iTru7_plate_96-1 REPLICATE,11505512.0,A01,iTru7_101_01,CAA GCA GAA GAC GGC ATA CGA GAT GGT AAC GTG TG...,231436232.0,0.0,14667.0,0.60,1.25,18.0,0.0,0.0,478700.0,68.0,A,1,A,1
1,iTru7_plate_96-1 REPLICATE,11505512.0,A02,iTru7_101_02,CAA GCA GAA GAC GGC ATA CGA GAT CAA CAC AGG TG...,231436231.0,0.0,14605.0,0.60,1.25,18.0,0.0,0.0,480100.0,68.0,A,2,A,3
2,iTru7_plate_96-1 REPLICATE,11505512.0,A03,iTru7_101_03,CAA GCA GAA GAC GGC ATA CGA GAT ACA CCT CAG TG...,231436229.0,0.0,14556.0,0.59,1.25,18.0,0.0,0.0,473900.0,68.0,A,3,A,5
3,iTru7_plate_96-1 REPLICATE,11505512.0,A04,iTru7_101_04,CAA GCA GAA GAC GGC ATA CGA GAT CAT GGA TCG TG...,231436228.0,0.0,14627.0,0.60,1.25,18.0,0.0,0.0,476200.0,68.0,A,4,A,7
4,iTru7_plate_96-1 REPLICATE,11505512.0,A05,iTru7_101_05,CAA GCA GAA GAC GGC ATA CGA GAT TGA TAG GCG TG...,231436226.0,0.0,14667.0,0.60,1.25,18.0,0.0,0.0,478500.0,68.0,A,5,A,9


In [101]:
# cut down to columns used for sample sheet: i5

idt_map_i5['Primer Index Sequence'] = idt_map_i5['Sequence'].str.replace(' ','').str[29:37]
idt_map_i5['count'] = idt_map_i5.index + 1

i5 = idt_map_i5.loc[:,['count', 'Sequence Name', 'Primer Index Sequence', 'Plate Name', 'origin row', 'origin col', 'condensed row', 'condensed col']]
i5['MiSeq & HiSeq Index Sequence'] = i5['Primer Index Sequence']
i5['MiniSeq & NextSeq Index Sequence'] = i5['Primer Index Sequence'].apply(rc)

i5 = i5.loc[:,['count', 'Sequence Name', 'Primer Index Sequence', 'MiSeq & HiSeq Index Sequence', 'MiniSeq & NextSeq Index Sequence',
         'Plate Name', 'origin row', 'origin col', 'condensed row', 'condensed col']]

i5.rename(columns={'Sequence Name': 'Primer Name', 'condensed row': 'Row', 'condensed col': 'Col'}, inplace=True)

i5.head()

,count,Primer Name,Primer Index Sequence,MiSeq & HiSeq Index Sequence,MiniSeq & NextSeq Index Sequence,Plate Name,origin row,origin col,Row,Col
0,1,iTru5_01_A,ACCGACAA,ACCGACAA,TTGTCGGT,iTru5_plate_96-1 REPLICATE,A,1,A,1
1,2,iTru5_02_A,CTTCGCAA,CTTCGCAA,TTGCGAAG,iTru5_plate_96-1 REPLICATE,A,2,A,3
2,3,iTru5_03_A,AACACCAC,AACACCAC,GTGGTGTT,iTru5_plate_96-1 REPLICATE,A,3,A,5
3,4,iTru5_04_A,CGTATCTC,CGTATCTC,GAGATACG,iTru5_plate_96-1 REPLICATE,A,4,A,7
4,5,iTru5_05_A,GGTACGAA,GGTACGAA,TTCGTACC,iTru5_plate_96-1 REPLICATE,A,5,A,9


In [102]:
# cut down to columns used for sample sheet: i5=7

idt_map_i7['Primer Index Sequence'] = idt_map_i7['Sequence'].str.replace(' ','').str[24:32].apply(rc)
idt_map_i7['count'] = idt_map_i7.index + 1

i7 = idt_map_i7.loc[:,['count', 'Sequence Name', 'Primer Index Sequence', 'Plate Name', 'origin row', 'origin col', 'condensed row', 'condensed col']]

i7 = i7.loc[:,['count', 'Sequence Name', 'Primer Index Sequence',
         'Plate Name', 'origin row', 'origin col', 'condensed row', 'condensed col']]

i7.rename(columns={'Sequence Name': 'Primer Name', 'condensed row': 'Row', 'condensed col': 'Col'}, inplace=True)

i7.head()

,count,Primer Name,Primer Index Sequence,Plate Name,origin row,origin col,Row,Col
0,1,iTru7_101_01,ACGTTACC,iTru7_plate_96-1 REPLICATE,A,1,A,1
1,2,iTru7_101_02,CTGTGTTG,iTru7_plate_96-1 REPLICATE,A,2,A,3
2,3,iTru7_101_03,TGAGGTGT,iTru7_plate_96-1 REPLICATE,A,3,A,5
3,4,iTru7_101_04,GATCCATG,iTru7_plate_96-1 REPLICATE,A,4,A,7
4,5,iTru7_101_05,GCCTATCA,iTru7_plate_96-1 REPLICATE,A,5,A,9


In [103]:
# write to excel

writer = pd.ExcelWriter('./test_data/iTru/new_iTru_index_list_Dec2017.xlsx')
i5.to_excel(writer,'iTru5')
i7.to_excel(writer,'iTru7')
writer.save()

In [104]:
# i5 = pd.read_excel('./test_data/iTru/new_iTru_index_list.xlsx',
#                sheetname='iTru5')

# i7 = pd.read_excel('./test_data/iTru/new_iTru_index_list.xlsx',
#                sheetname='iTru7')

In [105]:
i5.head()

,count,Primer Name,Primer Index Sequence,MiSeq & HiSeq Index Sequence,MiniSeq & NextSeq Index Sequence,Plate Name,origin row,origin col,Row,Col
0,1,iTru5_01_A,ACCGACAA,ACCGACAA,TTGTCGGT,iTru5_plate_96-1 REPLICATE,A,1,A,1
1,2,iTru5_02_A,CTTCGCAA,CTTCGCAA,TTGCGAAG,iTru5_plate_96-1 REPLICATE,A,2,A,3
2,3,iTru5_03_A,AACACCAC,AACACCAC,GTGGTGTT,iTru5_plate_96-1 REPLICATE,A,3,A,5
3,4,iTru5_04_A,CGTATCTC,CGTATCTC,GAGATACG,iTru5_plate_96-1 REPLICATE,A,4,A,7
4,5,iTru5_05_A,GGTACGAA,GGTACGAA,TTCGTACC,iTru5_plate_96-1 REPLICATE,A,5,A,9


In [106]:
i7.head()

,count,Primer Name,Primer Index Sequence,Plate Name,origin row,origin col,Row,Col
0,1,iTru7_101_01,ACGTTACC,iTru7_plate_96-1 REPLICATE,A,1,A,1
1,2,iTru7_101_02,CTGTGTTG,iTru7_plate_96-1 REPLICATE,A,2,A,3
2,3,iTru7_101_03,TGAGGTGT,iTru7_plate_96-1 REPLICATE,A,3,A,5
3,4,iTru7_101_04,GATCCATG,iTru7_plate_96-1 REPLICATE,A,4,A,7
4,5,iTru7_101_05,GCCTATCA,iTru7_plate_96-1 REPLICATE,A,5,A,9


In [107]:
combos = 384 * 384
i5_name = [''] * combos
i7_name = [''] * combos
i5_seq = [''] * combos
i7_seq = [''] * combos
i57_seq = [''] * combos
i5_well = [''] * combos
i7_well = [''] * combos
i5_plate = ['iTru5_plate'] * combos
i7_plate = ['iTru7_plate'] * combos
idx = [np.nan] * combos

i = 0
for i7_offset in range(384):
    for x in range(384):
        i5_idx = x
        i7_idx = (x + i7_offset) % 384
        idx[i] = i
        i5_name[i] = i5.iloc[i5_idx]['Primer Name']
        i7_name[i] = i7.iloc[i7_idx]['Primer Name']
        i5_seq[i] =  i5.iloc[i5_idx]['Primer Index Sequence']
        i7_seq[i] = i7.iloc[i7_idx]['Primer Index Sequence']
        i57_seq[i] = i5_seq[i] + i7_seq[i]
        i5_well[i] =  '{0}{1}'.format(i5.iloc[i5_idx]['Row'],i5.iloc[i5_idx]['Col'])
        i7_well[i] = '{0}{1}'.format(i7.iloc[i7_idx]['Row'],i7.iloc[i7_idx]['Col'])
        i += 1

index_combo_df = pd.DataFrame(np.array([idx, i57_seq, i5_name, i5_seq, i5_well, i5_plate, i7_name, i7_seq, i7_well, i7_plate]).T,
                              columns = ['index combo','index combo seq','i5 name','i5 sequence','i5 well','i5 plate',
                                         'i7 name','i7 sequence','i7 well','i7 plate'])

In [108]:
index_combo_df

,index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate
0,0,ACCGACAAACGTTACC,iTru5_01_A,ACCGACAA,A1,iTru5_plate,iTru7_101_01,ACGTTACC,A1,iTru7_plate
1,1,CTTCGCAACTGTGTTG,iTru5_02_A,CTTCGCAA,A3,iTru5_plate,iTru7_101_02,CTGTGTTG,A3,iTru7_plate
2,2,AACACCACTGAGGTGT,iTru5_03_A,AACACCAC,A5,iTru5_plate,iTru7_101_03,TGAGGTGT,A5,iTru7_plate
3,3,CGTATCTCGATCCATG,iTru5_04_A,CGTATCTC,A7,iTru5_plate,iTru7_101_04,GATCCATG,A7,iTru7_plate
4,4,GGTACGAAGCCTATCA,iTru5_05_A,GGTACGAA,A9,iTru5_plate,iTru7_101_05,GCCTATCA,A9,iTru7_plate
5,5,CGATCGATAACAACCG,iTru5_06_A,CGATCGAT,A11,iTru5_plate,iTru7_101_06,AACAACCG,A11,iTru7_plate
6,6,AAGACACCACTCGTTG,iTru5_07_A,AAGACACC,A13,iTru5_plate,iTru7_101_07,ACTCGTTG,A13,iTru7_plate
7,7,CATCTGCTCCTATGGT,iTru5_08_A,CATCTGCT,A15,iTru5_plate,iTru7_101_08,CCTATGGT,A15,iTru7_plate
8,8,CTCTCAGATGTACACC,iTru5_09_A,CTCTCAGA,A17,iTru5_plate,iTru7_101_09,TGTACACC,A17,iTru7_plate
9,9,TCGTCTGAGTATGCTG,iTru5_10_A,TCGTCTGA,A19,iTru5_plate,iTru7_101_10,GTATGCTG,A19,iTru7_plate


In [109]:
len(set(index_combo_df['index combo seq']))

147456

In [110]:
index_combo_df.to_csv('./test_output/iTru/new_iTru_combos_Dec2017.csv', index=False)

In [111]:
# check against old list

old_index_combo_df = pd.read_csv('./test_output/iTru/new_iTru_combos.csv')

In [113]:
old_index_combo_df.head()

,index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate
0,0,ACCGACAAACGTTACC,iTru5_01_A,ACCGACAA,A1,iTru5_plate,iTru7_101_01,ACGTTACC,A1,iTru7_plate
1,1,AGTGGCAACTGTGTTG,iTru5_01_B,AGTGGCAA,B1,iTru5_plate,iTru7_101_02,CTGTGTTG,A2,iTru7_plate
2,2,CACAGACTTGAGGTGT,iTru5_01_C,CACAGACT,C1,iTru5_plate,iTru7_101_03,TGAGGTGT,A3,iTru7_plate
3,3,CGACACTTGATCCATG,iTru5_01_D,CGACACTT,D1,iTru5_plate,iTru7_101_04,GATCCATG,A4,iTru7_plate
4,4,GACTTGTGGCCTATCA,iTru5_01_E,GACTTGTG,E1,iTru5_plate,iTru7_101_05,GCCTATCA,A5,iTru7_plate


In [115]:
set(old_index_combo_df['index combo seq']) == set(index_combo_df['index combo seq'])

True